# Grab some data for a different project

In [2]:
# since this notebook lives in a sub-folder of the main project, I'll add the main folder to the python path
import sys
sys.path.append("../")
import tools.glooko as gl
import tools.sugar_plots as spl

# plotting etc
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib widget

import numpy as np

## Load CGM data

In [3]:
data_folder = r"../data"
df_cgm, df_bolus, df_basal, df_insulin, df_cgm_daily = gl.merge_data(data_folder)
df_cgm.sort_values(by="time", inplace=True)

# Add time zone info
df_cgm["time"] = df_cgm["time"].dt.tz_localize(
    'US/Eastern', nonexistent='shift_backward').reset_index(drop=True)

df_cgm.reset_index(drop=True, inplace=True)

df_cgm.head()

,time,bg,dayofyear,year,yearday
0,2023-03-11 19:33:00-05:00,247.0,18,2023,2023-018
1,2023-03-11 19:28:00-05:00,241.0,18,2023,2023-018
2,2023-03-11 19:23:00-05:00,232.0,18,2023,2023-018
3,2023-03-11 19:18:00-05:00,225.0,18,2023,2023-018
4,2023-03-11 19:13:00-05:00,219.0,18,2023,2023-018


## Load Pod Alarm data

In [13]:
import glob 

files = glob.glob(data_folder + '/**/alarms_data.csv', 
                  recursive = True) 

# Concatenate all files into a dataframe
df_alarms = (pd.concat((pd.read_csv(f, header=1) for f in files), 
                       ignore_index=True).drop_duplicates())

# Convert time to datetime format and add time zone information
df_alarms["time"] = (pd.to_datetime(df_alarms["Timestamp"])                       
                   .dt.tz_localize('US/Eastern', nonexistent='shift_backward')) 

df_alarms = df_alarms.sort_values(by="time").reset_index(drop=True)


# Flag for pod change (seems most reliable?)
pod_change_flag = 'Pod will expire at user set time'



## PICKLE

In [15]:
import pickle

# Get all the data I want to pickle together
glooko_data = (df_cgm,
               df_alarms,
               pod_change_flag)

# Pickle!
out_file_name = r"../../data/glooko_cgm_and_pump.pkl"
file = open(out_file_name, 'wb')

# dump information to that file
pickle.dump(glooko_data, file)

# close the file
file.close()